In [2]:
import numpy as np
import pylab as pl
import cosmocnc as cnc
import time
from astropy.io import fits
import os
import nemo as nm
import scipy.stats

2025-04-16 18:08:46.141939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744819726.234706 1523101 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744819726.259390 1523101 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


defaulting to  /rds-d4/user/iz221/hpc-work/cosmopower/
defaulting to  /home/anicola_hpc/projects/ACT_clustering/code/cosmocnc/cosmocnc/../
PATH_TO_CLASS_SZ_DATA not set. Setting it now...
Defaulting to: /home/anicola_hpc/class_sz_data_directory


2025-04-16 18:08:50.798355: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [2]:
datadir = '/lustre/scratch/data/anicola_hpc-ACT_clustering/data/DR5ClusterSearch/selFn'
datafile_rms = os.path.join(datadir, 'RMSTab_DES.fits')
datafile_Q = os.path.join(datadir, 'QFit.fits')

In [3]:
with fits.open(datafile_rms) as in_file:
    file_rms = in_file[1].data

In [4]:
tile_name = np.unique(file_rms['tileName'])
tile_name = np.array(tile_name, dtype = str) # Avoids potential chararray weirdness
QFit = nm.signals.QFit(QSource='fit', selFnDir=datadir,
                        tileNames = tile_name)
Nt = len(tile_name)
tname = np.array(file_rms['tileName'], dtype = str) # Avoids potential chararray weirdness

with fits.open(datafile_Q) as hdulist:
    data = hdulist[1].data
    tt500 = data.field("theta500Arcmin")

# reading in all Q functions
allQ = np.zeros((len(tt500), Nt))
for i in range(Nt):
    allQ[:, i] = QFit.getQ(tt500, tileName=tile_name[i])
assert len(tt500) == len(allQ[:, 0])

Q = allQ

In [5]:
dwnsmpl_bins = 49

In [6]:
noise = file_rms['y0RMS']

# print(self.noise.min(), self.noise.max()) # 5.5760333e-06 4.0576047e-05

skyfracs = file_rms['areaDeg2'] * np.deg2rad(1.) ** 2
print("Number of RMS values = {}.".format(skyfracs.size))
print("Down-sampling RMS and Q function using {} bins.".format(dwnsmpl_bins))
binned_stat = scipy.stats.binned_statistic(noise, skyfracs, statistic='sum',
                                            bins=dwnsmpl_bins)
binned_area = binned_stat[0]
binned_rms_edges = binned_stat[1]

bin_ind = np.digitize(noise, binned_rms_edges)
tiledict = dict(zip(tile_name, np.arange(tile_name.shape[0])))

Qdwnsmpld = np.zeros((Q.shape[0], dwnsmpl_bins))
tiles_dwnsmpld = {}

for i in range(dwnsmpl_bins):
    # print(i)
    tempind = np.where(bin_ind == i + 1)[0]
    if len(tempind) == 0:
        print('Found empty bin.')
        Qdwnsmpld[:, i] = np.zeros(Q.shape[0])
    else:
        # print('dowsampled rms bin ',i)
        temparea = skyfracs[tempind]
        # print('areas of tiles in bin',temparea)
        temptiles = tname[tempind]
        # print('names of tiles in bin',temptiles)
        for t in temptiles:
            tiles_dwnsmpld[t] = i

        test = [tiledict[key] for key in temptiles]
        Qdwnsmpld[:, i] = np.average(Q[:, test], axis=1, weights=temparea)

noise = 0.5*(binned_rms_edges[:-1] + binned_rms_edges[1:])
skyfracs = binned_area
Q = Qdwnsmpld
tiles_dwnsmpld = tiles_dwnsmpld

Number of RMS values = 16010.
Down-sampling RMS and Q function using 49 bins.


In [7]:
datafile_rms_dwsmpld = os.path.join(datadir, 'RMSTab_DES_dwsmpld_nbins={}.npz'.format(dwnsmpl_bins))
datafile_Q_dwsmpld = os.path.join(datadir, 'QFit_dwsmpld_nbins={}.npz'.format(dwnsmpl_bins))

In [8]:
np.savez(datafile_Q_dwsmpld, Q_dwsmpld=Qdwnsmpld, tt500=tt500)
np.savez(datafile_rms_dwsmpld, noise=noise, skyfracs=skyfracs)

In [9]:
np.sum(skyfracs)*(180/np.pi)**2

4608.488160967827

In [3]:
bias_vec = np.array([4.38, 5.22, 7.8, 1.91])
bias_std = np.array([0.67, 0.79, 1.3, 0.87])

bias_cov = np.diagflat(bias_std**2)

In [7]:
np.savetxt('/lustre/scratch/data/anicola_hpc-ACT_clustering/data/bias_DESxACT_vec_z-bin1-3.txt', bias_vec[:-1])
np.savetxt('/lustre/scratch/data/anicola_hpc-ACT_clustering/data/bias_DESxACT_cov_z-bin1-3.txt', bias_cov[:-1, :][:, :-1])